In [66]:
import os
import pickle
import numpy as np
import torch
import sys
sys.path.append('../inverse_rendering/')
machine_path = os.getcwd()
user_root_dir = '/'.join(machine_path.split('/')[:-2])
sys.path.insert(0,'%s/redner/'%user_root_dir)
import redner
import pyredner
print(pyredner.__file__)
pyredner.render_pytorch.print_timing = False
from general_imports import *

/net/storage001.ib.cluster/om2/user/smadan/redner/pyredner/__init__.py
/net/storage001.ib.cluster/om2/user/smadan/redner/pyredner/__init__.py
/net/storage001.ib.cluster/om2/user/smadan/training_scaffold_own/res/loader/multi_attribute_loader.py
/net/storage001.ib.cluster/om2/user/smadan/training_scaffold_own/res/loader
/net/storage001.ib.cluster/om2/user/smadan/training_scaffold_own/res/loader/loader.py
/net/storage001.ib.cluster/om2/user/smadan/training_scaffold_own/res/loader


In [67]:
results_folders = ["cma_adversarial_cam/neurips_results/%s"%i for i in os.listdir('cma_adversarial_cam/neurips_results/')]

In [68]:
def clone_scene_params(scene_params):
    original_scene_params = [0,0,0,0]
    original_scene_params[0] = scene_params[0]

    original_scene_params[1] = {}
    for key in scene_params[1].keys():
        original_scene_params[1][key] = scene_params[1][key].clone()

    original_scene_params[2] = {}
    for key in scene_params[2].keys():
        original_scene_params[2][key] = [i.clone() for i in scene_params[2][key]]

    original_scene_params[3] = scene_params[3]
    return original_scene_params

In [69]:
def get_final_params(initial_params, x):
    clone = clone_scene_params(initial_params)
    clone[1]['camera_position'] = torch.tensor(x[:3]).float()
    clone[1]['cam_look_at'] = torch.tensor(x[3:6]).float()
    clone[1]['cam_up'] = torch.tensor(x[6:9]/10).float()
    clone[1]['fov'] = torch.tensor(x[9]*10).float()
    return clone

In [70]:
def get_final_params_light(initial_params, x):
    clone = clone_scene_params(initial_params)
    clone[1]['camera_position'] = torch.tensor(x[:3]).float()
    clone[1]['cam_look_at'] = torch.tensor(x[3:6]).float()
    clone[1]['cam_up'] = torch.tensor(x[6:9]/10).float()
    clone[1]['fov'] = torch.tensor(x[9]*10).float()
    return clone

In [72]:
def calculate_distances(initial_params,final_params):
    pos_dist = torch.linalg.norm(initial_params[1]['camera_position'] - final_params[1]['camera_position'],1)/(3*4)*100
    look_dist = torch.linalg.norm(initial_params[1]['cam_look_at'] - final_params[1]['cam_look_at'],1)/(3*4*0.3)*100
    return pos_dist, look_dist

In [104]:
np.array(exp_info['es'].predictions) != exp_info['es'].correct_prediction

array([False, False, False, False, False, False, False, False, False,
        True])

In [107]:
# for result_folder in results_folders:
#     exp_files = ['%s/%s'%(result_folder,i) for i in os.listdir(result_folder) if i.endswith('.p')]
#     err_exp = 0
#     total_exp = 0
#     for exp_file in exp_files:
#         total_exp += 1
#         with open(exp_file,'rb') as F:
#             exp_info = pickle.load(F)
#             errors = np.array(exp_info['es'].predictions) != exp_info['es'].correct_prediction
#             if len(errors)>0:
#                 err_exp += 1
#     print(err_exp/total_exp)

In [96]:
meta_all_cd = []
meta_all_ld = []
for result_folder in results_folders:
    exp_files = ['%s/%s'%(result_folder,i) for i in os.listdir(result_folder) if i.endswith('.p')]
    all_cd = []
    all_ld = []
    for exp_file in exp_files:
        with open(exp_file,'rb') as F:
            exp_info = pickle.load(F)
            initial = exp_info['scene_params']
            adv_positions = np.where(np.array(exp_info['es'].predictions) != exp_info['es'].correct_prediction)[0]
            if len(adv_positions)>0:
                adv_position = adv_positions[0]
                adv_setting = exp_info['es'].prediction_settings[adv_position]
                final = get_final_params(initial, adv_setting)
                cd, ld = calculate_distances(initial, final)
                all_cd.append(cd)
                all_ld.append(ld)
                
    print(result_folder, "%s +- %s"%(np.mean(all_cd), np.std(all_cd)), "%s +- %s"%(np.mean(all_ld), np.std(all_ld)))
    meta_all_cd.append(np.mean(all_cd))
    meta_all_ld.append(np.mean(all_ld))

cma_adversarial_cam/neurips_results/resnet18_v7_normalized_final 1.7481127 +- 1.7060299 5.833493 +- 5.902203
cma_adversarial_cam/neurips_results/resnet18_pretrained_v7_normalized_final 1.7129765 +- 1.7403842 5.6361227 +- 4.6827474
cma_adversarial_cam/neurips_results/resnet18_antialiased_v7_normalized_final 3.293728 +- 3.8250768 10.467619 +- 10.664044
cma_adversarial_cam/neurips_results/resnet18_v7_truly_shift_invariant_normalized_final 2.2782829 +- 2.3509247 6.787909 +- 6.428416
cma_adversarial_cam/neurips_results/train_v7_transformer_2_final 1.3181101 +- 0.8682036 4.29823 +- 2.3946288
cma_adversarial_cam/neurips_results/train_v7_transformer_2_deit 1.2455395 +- 0.78311616 4.4083495 +- 3.2457292
cma_adversarial_cam/neurips_results/train_v7_transformer_2_deit_distilled 1.2083895 +- 0.68837905 4.2269783 +- 2.8569345
cma_adversarial_cam/neurips_results/resnet18_v7_40_final 1.8336893 +- 1.3259414 6.5231795 +- 5.684479
cma_adversarial_cam/neurips_results/resnet18_pretrained_v7_40_normalized_

In [100]:
# np.mean(all_cd)

In [101]:
# np.std(all_cd)

In [76]:
import torch
import numpy as np

In [77]:
torch.FloatTensor([1,1,1])

tensor([1., 1., 1.])

In [84]:
def calculate_distances_simple(param_1,param_2, max_range):
    return torch.linalg.norm(torch.FloatTensor(param_1) - torch.FloatTensor(param_2),1)/(3*max_range)*100

In [85]:
calculate_distances_simple([-1.15,0.97,-0.79],[-0.95,1.3,-0.89],4)

tensor(5.2500)

In [86]:
calculate_distances_simple([-1.15,0.97,-0.79],[-0.95,1.3,-0.89],1.2)

tensor(17.5000)

In [89]:
calculate_distances_simple([0.38,0.9,1.29],[0.37,1,1.32],4)

tensor(1.1667)

In [90]:
calculate_distances_simple([0.22,0.43,0.3],[0.16,0.23,0.36],1.2)

tensor(8.8889)

In [91]:
calculate_distances_simple([1.44, -0.1,0.78],[1.48,-0.09,0.84],4)

tensor(0.9167)

In [92]:
calculate_distances_simple([0.15,0.04,0.1],[0.13,-0.03,0.13],1.2)

tensor(3.3333)

In [93]:
calculate_distances_simple([-1.32,-0.65,-0.68],[-1.25,-0.72,-0.69],4)

tensor(1.2500)

In [94]:
calculate_distances_simple([0.26,0.37,0.04],[0.3,0.33,0.06],1.2)

tensor(2.7778)